# Önemli Uyarı !!!
### XGBClassifier sınıfını kullanmak için py-xgboost paketini yüklemiş olmalısınız.

In [27]:
#! pip install xgboost

In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

%matplotlib inline

In [29]:
# veri kaynağı: http://veribilimi.co/data/iris.csv
df = pd.read_csv("D:/Datasets/SosyalMedyaReklamKampanyasi.csv")

# 1. Veri Keşfi

In [30]:
df.head()

KullaniciID Cinsiyet  Yas  TahminiMaas  SatinAldiMi
0     15624510    Erkek   19        19000            0
1     15810944    Erkek   35        20000            0
2     15668575    Kadın   26        43000            0
3     15603246    Kadın   27        57000            0
4     15804002    Erkek   19        76000            0

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
KullaniciID    400 non-null int64
Cinsiyet       400 non-null object
Yas            400 non-null int64
TahminiMaas    400 non-null int64
SatinAldiMi    400 non-null int64
dtypes: int64(4), object(1)
memory usage: 15.7+ KB


In [32]:
df.describe()

KullaniciID         Yas    TahminiMaas  SatinAldiMi
count  4.000000e+02  400.000000     400.000000   400.000000
mean   1.569154e+07   37.655000   69742.500000     0.357500
std    7.165832e+04   10.482877   34096.960282     0.479864
min    1.556669e+07   18.000000   15000.000000     0.000000
25%    1.562676e+07   29.750000   43000.000000     0.000000
50%    1.569434e+07   37.000000   70000.000000     0.000000
75%    1.575036e+07   46.000000   88000.000000     1.000000
max    1.581524e+07   60.000000  150000.000000     1.000000

In [33]:
# Veri keşfi ile başka birçok groupby, count_values vb. sorgular kullanılabilir. Grafikler çizilip incelenebilr.

# 2. Veri Temizliği ve Ön Hazırlığı

In [34]:
# Veri önceden temizlenmiş olduğundan bu aşamada birşey yapmıyoruz.Boşluk yok aykırı değer yok.

## 2.1. Veriyi X,y olarak bölme (nitelikler matrisi ve hedef değişken - dikey bölme)

In [35]:
# Hedef değişken en sonda olduğu için ve sadece nümerik nitelikleri kullanacağımız için
# baştan (0) son indekse kadar tüm nitelikleri alıyoruz
X = df.iloc[:,1:-1].values
X.shape

(400, 3)

In [36]:
# Kontrol
X[:3]

array([['Erkek', 19, 19000],
       ['Erkek', 35, 20000],
       ['Kadın', 26, 43000]], dtype=object)

In [37]:
# En son indeks hedef değiken (output)
y = df.iloc[:,-1].values
y.shape

(400,)

In [38]:
# Kontrol
y[:10]

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0], dtype=int64)

## 2.2. Kategorik Nitelik Dönüştürme

### 2.2.1. Girdi Nitelikler için LabelEncoder

In [39]:
# Her bir kategorik nitelik için ayrı bir LabelEncoder nesnesi yaratalım. 
# Tek bir nesne ile de yapabiliriz ancak bu durumda orjinal değerlere dönmek mümkün olmaz.
from sklearn.preprocessing import LabelEncoder
labelencoder_cinsiyet = LabelEncoder()

In [40]:
# Her bir kategorik nitelik için oluşturduğumuz LabelEncoder nesnelerini (makine) eğitip sonucu dönüştürüp string değerler
# ile dğiştiriyoruz. Artık X içinde string değer kalmıyor. Onların yerine nümerik karşılıkları var. Bu nümerik değerlerin
# orjinal veride hangi string değere karşılık geldiğini ilgili LabelEncoder nesnesi biliyor.
X[:,0] = labelencoder_cinsiyet.fit_transform(X[:,0])

### 2.2.2. Hedef değişken için LabelEncoder

In [41]:
# Hedef değişken 0 ve 1 olarak kodlanmış

### 2.2.3. One Hot Encoder - ColumnTransformer

In [42]:
from sklearn.preprocessing import OneHotEncoder

In [43]:
onehotencoder = OneHotEncoder(handle_unknown='ignore', categories='auto')

In [44]:
# Birden sütunu ilgilendiren dönüşümleri kolay yapabilmek için kullanışlı bir sınıf (Estimator kökenli)
# Biz burada sadece OneHotEncoder için kullanacağız.
from sklearn.compose import ColumnTransformer

In [45]:
# one_hot_encode_with_ct bizim kendi verdiğiiz isimdir. onehotencoder'ı yukarıda yarattık. [0,1,2,3,4,5,6,7,8] ise dönüşecek
# niteliklerin indeks değerleridir.
ct = ColumnTransformer(
    [('one_hot_encode_with_ct', onehotencoder, [0]),], 
    remainder='passthrough'
)

In [46]:
# ColumnTransformer nesnesi ile onehotencoder nesnesini eğitip gerekli dönüşümleri gerçekleştirip X'e tekrar atayalım.
# Burada X'in boyutu(shape) değişecektir.
X = ct.fit_transform(X)

In [47]:
# Boyut değişmiş olmalı. Çnkü gölge değişkenler eklendi
X.shape

(400, 4)

In [48]:
X[:3,:]

array([[1.0, 0.0, 19, 19000],
       [1.0, 0.0, 35, 20000],
       [0.0, 1.0, 26, 43000]], dtype=object)

## 2.3. Feature Scaling (Standardizasyon)

In [49]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X = sc_X.fit_transform(X)

## 2.4. Veriyi eğitim ve test setlerine bölme (yatay bölme)

In [50]:
from sklearn.model_selection import train_test_split

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=142)

In [52]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(300, 4)
(300,)
(100, 4)
(100,)


# 3. Model Oluşturma ve Eğitim

In [53]:
from xgboost import XGBClassifier

In [54]:
classifier = XGBClassifier(random_state=142)

In [55]:
classifier.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=142,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

# 4. Predictions

In [56]:
y_pred = classifier.predict(X_test)

## 4.1. Tahmin sonuçlarını numpy.ndarray'den dataframe yapma (İncelemek daha Kolay)

In [57]:
predictions = pd.DataFrame(data=y_pred,    # values
                index=range(len(y_pred)),    # 1st column as index
                   columns=['y_pred'])  # 1st row as the column names

# Sadece y_pred'den oluşan df'e test(gerçek) y_test'i sütun olarak ekleme
predictions['y_test'] = y_test
predictions.head()

y_pred  y_test
0       1       1
1       1       1
2       0       0
3       1       0
4       1       0

# 5. Model Değerlendirme

## 5.1. Hata Matrisi (Confusion Matrix)

In [58]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true=y_test, y_pred=y_pred)
print(cm)

[[58  9]
 [ 4 29]]


## 5.2. Sınıflandırma Performansı Değerlendirme (Accuracy)

In [59]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy: % {:10.2f}".format(accuracy*100)) 
# :10.2f ifadesi ondalık kısımda iki basamak göstermek için

Accuracy: %      87.00


# 6. Modeli Saklama ve Tekrar Kullanma

In [60]:
# Saklamadan önce tüm veri seti ile modeli en iyi hiperparametrelerle tekrar eğitelim

## 6.1. Tüm veri ile modeli son olarak eğitme

In [61]:
# Boş bir sınıflandırıcı (SVC) nesnesi oluşturalım
classifier_last = XGBClassifier(random_state=142)

In [62]:
classifier_last.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=142,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

## 8.2. Modeli kaydetme

In [64]:
import joblib

In [65]:
joblib.dump(classifier_last, 
"D:/OneDrive/egitim/verilen/machine-learning-with-python/10_classification/saved_models/07.xgboost_SosyalMedyaReklamKampanyası_data.pkl")

['D:/OneDrive/egitim/verilen/machine-learning-with-python/10_classification/saved_models/07.xgboost_SosyalMedyaReklamKampanyası_data.pkl']

## 8.3. Model Okuma

In [66]:
classifier_last_loaded = joblib.load(
"D:/OneDrive/egitim/verilen/machine-learning-with-python/10_classification/saved_models/07.xgboost_SosyalMedyaReklamKampanyası_data.pkl")

In [67]:
classifier_last_loaded.get_params

<bound method XGBModel.get_params of XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=nan, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=142,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)>

In [ ]:
# Ödev-25